In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
#data=pd.read_csv("../data/Raw Data_GeneSpring.txt",header=None)

In [3]:
with open('../data/Raw Data_GeneSpring.txt') as fp:
    # 1. iterate over file line-by-line
    # 2. strip line of newline symbols
    # 3. split line by spaces into list (of number strings)
    # 4. convert number substrings to int values
    # 5. convert map object to list
    data = [list(map(str, line.strip().split("\t"))) for line in fp]

In [4]:
data=np.array(data)

In [5]:
#data[2][48]

In [6]:
#Gene Symbol
#for Gene Symbol
gene_symbol=[]
for i in range(1,len(data)):
        if(len(data[i])>49):
            gene_symbol.append(data[i][49])
        else:
            gene_symbol.append('NA')

In [7]:
#gene_symbol

In [8]:
for i in range(1,len(data)):
    for j in range(len(data[1])):
        if(j!=0):
            data[i][j]=float(data[i][j])

In [9]:
tmp=[]
for i in range(1,len(data)):
    tmp.append(data[i][3])

In [10]:
"""import matplotlib.pyplot as plt
plt.yscale('log')
plt.hist(tmp,bins=40,alpha=0.75,rwidth=0.3)"""

"import matplotlib.pyplot as plt\nplt.yscale('log')\nplt.hist(tmp,bins=40,alpha=0.75,rwidth=0.3)"

In [11]:
# Male Non Smokers - (106-117)
# Male Smokers - (118-129)
# FeMale Non Smokers - (130-141)
# FeMale Smokers - (142-153)

In [12]:
#non_smokers_male
non_smokers_male=[[0.0]*12 for p in range(1,len(data))]
for i in range(1,len(data)):
    for j in range(1,13):
        non_smokers_male[i-1][j-1]=data[i][j]

#smokers_male
smokers_male=[[0.0]*12 for p in range(1,len(data))]
x=-1
for i in range(1,len(data)):
    x=x+1
    y=0
    for j in range(13,25):
        smokers_male[x][y]=data[i][j]
        y=y+1
        
#non_smokers_female
non_smokers_female=[[0.0]*12 for p in range(1,len(data))]
x=-1
for i in range(1,len(data)):
    x=x+1
    y=0
    for j in range(25,37):
        non_smokers_female[x][y]=data[i][j]
        y=y+1
        

#smokers_female
smokers_female=[[0.0]*12 for p in range(1,len(data))]
x=-1
for i in range(1,len(data)):
    x=x+1
    y=0
    for j in range(37,49):
        smokers_female[x][y]=data[i][j]
        y=y+1
          
            


In [13]:
male=np.hstack((non_smokers_male,smokers_male))
female=np.hstack((non_smokers_female,smokers_female))

In [14]:
all_data=np.hstack((male,female))

### Main

In [15]:
# Male Non Smokers - (106-117)
# Male Smokers - (118-129)
# FeMale Non Smokers - (130-141)
# FeMale Smokers - (142-153)
#matrix=[male_mean,female_mean,non_smoker_mean,smoker_mean]
A_null=[[0]*4 for p in range(48)]
for i in range(12):
    A_null[i][0]=1
    A_null[i][2]=1
    
for i in range(12,24):
    A_null[i][0]=1
    A_null[i][3]=1

for i in range(24,36):
    A_null[i][1]=1
    A_null[i][2]=1
    
for i in range(36,48):
    A_null[i][1]=1
    A_null[i][3]=1
    

In [16]:
#len(A_null)

In [17]:
#A_null

In [18]:
#matrix=[male_ns_mean,male_s_mean,female_ns_mean,female_s_mean]
A=[[0]*4 for p in range(48)]
for i in range(12):
    A[i][0]=1
    
for i in range(12,24):
    A[i][1]=1

for i in range(24,36):
    A[i][2]=1
    
for i in range(36,48):
    A[i][3]=1

In [19]:
#A

In [20]:
A=np.matrix(A)
A_null=np.matrix(A_null)
r1=np.linalg.matrix_rank(A_null)
r2=np.linalg.matrix_rank(A)

In [21]:
#r1

In [22]:
#r2

In [23]:
#Calculation of F statistics
n=48
I=np.identity(48)
F=[]
for i in range(len(all_data)):
   
    h=all_data[i]
    tmp1=np.matmul(h.T, I - (np.matmul(np.matmul(A_null,np.linalg.pinv(np.matmul(A_null.T,A_null))),A_null.T)))
    tmp2=np.matmul(h.T, I - (np.matmul(np.matmul(A,np.linalg.pinv(np.matmul(A.T,A))),A.T)))
    if(np.matmul(tmp2,h)==0):
        f_stats= ((np.matmul(tmp1,h)/0.0000000000000000001)-1)*(n-r2)/(r2-r1)
        F.append(f_stats)
    else:
        f_stats= ((np.matmul(tmp1,h)/np.matmul(tmp2,h))-1)*(n-r2)/(r2-r1)
        F.append(f_stats)
    

In [24]:
F=np.array(F)

In [25]:
F=F.tolist()

In [26]:
#len(F)

In [27]:
for i in range(len(F)):
    F[i]=F[i][0]

In [28]:
dfd=48-r2
dfn=r2-r1
p_score=1-stats.f.cdf(F,dfn,dfd)

In [29]:
print("p-Values Generated")

p-Values Generated


In [30]:
import matplotlib.pyplot as plt
plt.hist(p_score,bins=20)
plt.savefig("histogram.png")
print("Histogram for p-values saved")

Histogram for p-values saved


In [31]:
count=0
dict_genes={}
genes_list=[]
genes_symbol_list=[]
for i in range(len(p_score)):
    if(p_score[i]<0.05):
        count=count+1
        dict_genes[i]=[data[i][0],p_score[i][0]]
        genes_list.append(data[i][0])
        if(gene_symbol[i]!='NA'):
            genes_symbol_list.append(gene_symbol[i])
#print(count)
    

In [32]:
with open("genes_probe_list.txt", "w") as txt_file:
    for line in genes_list:
        txt_file.write("".join(line) + "\n")

In [33]:
with open("genes_symbol_list.txt", "w") as txt_file:
    for line in genes_symbol_list:
        txt_file.write("".join(line) + "\n")
print("Genes Symbol list file generated")

Genes Symbol list file generated


In [34]:
#genes_list

In [35]:
#dict_genes

In [36]:
#p_score[1][0]

In [37]:
import operator
sorted_dict = sorted(dict_genes.values(), key=operator.itemgetter(1), reverse=False)

In [38]:
#sorted_dict

### FDR 

In [39]:
#Let n0=0.5
q=0.05
n0=811*0.2
ps=[]
for i in range(len(p_score)):
    ps.append(p_score[i][0])

In [40]:
ps=np.sort(ps)

In [41]:
#ps

In [42]:
# count the non-nan values
#non_nans = (~np.isnan(ps)).sum(0)

In [43]:
c=0
for i in range(0,len(ps)):
    if((n0*ps[i]/(i+1))<=q):
        c=c+1
    

In [44]:
#c

### Find Intersection  (1)

In [45]:
#All Gene Symbols
#genes_symbol_list

In [46]:
with open('../data/XenobioticMetabolism1.txt') as fp:
    next(fp)
    next(fp)
    data_f = [list(map(str, line.strip().split("\t"))) for line in fp]

In [47]:
check=[]
for i in range(len(data_f)):
    check.append(data_f[i][0])

In [48]:
count=0
intersection1=[]
for p in genes_symbol_list:
    for q in check:
        if(p==q):
            intersection1.append(p)
            count=count+1

In [49]:
#count

In [50]:
print("Genes Symbol Intersetion with Xenobiotic: ", intersection1)
print("Count ::", len(intersection1) )

Genes Symbol Intersetion with Xenobiotic:  ['SULT1A1', 'AOC2', 'CYP2S1', 'AADAC', 'HNF4A', 'AS3MT']
Count :: 6


### Find Intersection (2) 

In [51]:
with open('../data/FreeRadicalResponse.txt') as fp:
    next(fp)
    next(fp)
    data_f = [list(map(str, line.strip().split("\t"))) for line in fp]

In [52]:
#len(data)

In [53]:
check=[]
for i in range(len(data_f)):
    check.append(data_f[i][0])
    
    

In [54]:
count=0
intersection2=[]
for p in genes_symbol_list:
    for q in check:
        if(p==q):
            intersection2.append(p)
            count=count+1

In [55]:
#count

In [56]:
#intersection2

### Find Intersection (3) 

In [57]:
with open('../data/DNARepair1.txt') as fp:
    next(fp)
    next(fp)
    data_f = [list(map(str, line.strip().split("\t"))) for line in fp]

In [58]:
check=[]
for i in range(len(data_f)):
    check.append(data_f[i][0])

In [59]:
count=0
intersection3=[]
for p in genes_symbol_list:
    for q in check:
        if(p==q):
            intersection3.append(p)
            count=count+1

In [60]:
#count

In [61]:
print("Genes Symbol Intersetion with DNA Repair:", intersection3)
print("Count ::", len(intersection3) )

Genes Symbol Intersetion with DNA Repair: ['PNKP']
Count :: 1


### Find Intersection (4) 

In [62]:
with open('../data/NKCellCytotoxicity.txt') as fp:
    next(fp)
    next(fp)
    data_f = [list(map(str, line.strip().split("\t"))) for line in fp]

In [63]:
#len(data_f)

In [64]:
check=[]
for i in range(len(data_f)):
    check.append(data_f[i][0])

In [65]:
count=0
intersection4=[]
for p in genes_symbol_list:
    for q in check:
        if(p==q):
            intersection4.append(p)
            count=count+1

In [66]:
#count

In [67]:
print("Genes Symbol Intersetion with NKCellCytotoxicity : ", intersection4)
print("Count ::", len(intersection4) )

Genes Symbol Intersetion with NKCellCytotoxicity :  ['IFNG', 'KLRC2', 'PTPN6', 'HLA-C', 'PRF1', 'HLA-E', 'HLA-G']
Count :: 7


### Grouping lists 

In [68]:
female_s_up=[]
female_s_down=[]
male_s_up=[]
male_s_down=[]

### Grouping (1)

In [69]:
rows=[]
tmp=[]
for i in range(len(gene_symbol)):
    for j in range(len(intersection1)):
        if(gene_symbol[i]==intersection1[j]):
            #print(intersection1[j])
            tmp.append(intersection1[j])
            rows.append(i)
            break 

In [70]:
#rows

In [71]:
#tmp

In [82]:
print("\nGrouping for Xenobiotic")
print("Probe-Name","Gene-Symbol","Male-NS","Male-S","Female-NS","Female-S")
for i in rows:
    male_ns=all_data[i,0:12]
    male_s=all_data[i,12:24]
    female_ns=all_data[i,24:36]
    female_s=all_data[i,36:48]
    male_ns_mean=np.mean(male_ns)
    male_s_mean=np.mean(male_s)
    female_ns_mean=np.mean(female_ns)
    female_s_mean=np.mean(female_s)
    if(male_ns_mean<male_s_mean):
        male_s_up.append(gene_symbol[i])
    if(male_ns_mean>male_s_mean):
        male_s_down.append(gene_symbol[i])
    if(female_ns_mean<female_s_mean):
        female_s_up.append(gene_symbol[i])
    if(female_ns_mean>female_s_mean):
        female_s_down.append(gene_symbol[i])
    
    print(data[i+1][0],gene_symbol[i],male_ns_mean,male_s_mean,female_ns_mean,female_s_mean)


Grouping for Xenobiotic
Probe-Name Gene-Symbol Male-NS Male-S Female-NS Female-S
A_23_P151294 IFNG 5.915514025 6.710654066666668 6.606077316666666 5.854469466666665
A_23_P22232 KLRC2 9.717394458333333 10.676192666666667 10.301468541666667 9.549306850000002
A_23_P113716 HLA-C 18.476547083333333 18.442838999999996 18.48375175 18.575801916666666
A_24_P936272 HLA-C 15.186137666666667 14.983310166666664 14.905019250000002 15.305031708333336
A_23_P162486 PTPN6 12.292167374999998 11.98547275 11.932862041666667 12.233174083333331
A_23_P70539 HLA-C 16.353414833333332 16.180303000000002 16.022591416666668 16.37197258333333
A_23_P95917 HLA-C 17.984208916666663 17.97244416666667 17.967677916666666 18.203733583333335
A_23_P1473 PRF1 10.366630999999998 10.682892208333334 10.011584791666667 9.422278416666666
A_24_P298409 HLA-C 17.93432891666667 17.873328166666667 17.885603833333334 18.0351685
A_24_P326082 HLA-E 17.694454 17.707546583333336 17.5425235 17.80270308333333
A_23_P30848 HLA-E 16.1834796666

### Grouping (2)

In [73]:
rows=[]
tmp=[]
for i in range(len(gene_symbol)):
    for j in range(len(intersection2)):
        if(gene_symbol[i]==intersection2[j]):
            #print(intersection1[j])
            tmp.append(intersection2[j])
            rows.append(i)
            break 

In [74]:
#rows

### Grouping (3) 

In [75]:
rows=[]
tmp=[]
for i in range(len(gene_symbol)):
    for j in range(len(intersection3)):
        if(gene_symbol[i]==intersection3[j]):
            #print(intersection1[j])
            tmp.append(intersection3[j])
            rows.append(i)
            break 

In [76]:
#rows

In [81]:
print("\nGrouping for DNA Repair")
print("Probe-Name","Gene-Symbol","Male-NS","Male-S","Female-NS","Female-S")
for i in rows:
    male_ns=all_data[i,0:12]
    male_s=all_data[i,12:24]
    female_ns=all_data[i,24:36]
    female_s=all_data[i,36:48]
    male_ns_mean=np.mean(male_ns)
    male_s_mean=np.mean(male_s)
    female_ns_mean=np.mean(female_ns)
    female_s_mean=np.mean(female_s)
    if(male_ns_mean<male_s_mean):
        male_s_up.append(gene_symbol[i])
    if(male_ns_mean>male_s_mean):
        male_s_down.append(gene_symbol[i])
    if(female_ns_mean<female_s_mean):
        female_s_up.append(gene_symbol[i])
    if(female_ns_mean>female_s_mean):
        female_s_down.append(gene_symbol[i])
    print(data[i+1][0],gene_symbol[i],male_ns_mean,male_s_mean,female_ns_mean,female_s_mean)


Grouping for DNA Repair
Probe-Name Gene-Symbol Male-NS Male-S Female-NS Female-S
A_23_P151294 IFNG 5.915514025 6.710654066666668 6.606077316666666 5.854469466666665
A_23_P22232 KLRC2 9.717394458333333 10.676192666666667 10.301468541666667 9.549306850000002
A_23_P113716 HLA-C 18.476547083333333 18.442838999999996 18.48375175 18.575801916666666
A_24_P936272 HLA-C 15.186137666666667 14.983310166666664 14.905019250000002 15.305031708333336
A_23_P162486 PTPN6 12.292167374999998 11.98547275 11.932862041666667 12.233174083333331
A_23_P70539 HLA-C 16.353414833333332 16.180303000000002 16.022591416666668 16.37197258333333
A_23_P95917 HLA-C 17.984208916666663 17.97244416666667 17.967677916666666 18.203733583333335
A_23_P1473 PRF1 10.366630999999998 10.682892208333334 10.011584791666667 9.422278416666666
A_24_P298409 HLA-C 17.93432891666667 17.873328166666667 17.885603833333334 18.0351685
A_24_P326082 HLA-E 17.694454 17.707546583333336 17.5425235 17.80270308333333
A_23_P30848 HLA-E 16.1834796666

### Grouping (4) 

In [78]:
rows=[]
tmp=[]
for i in range(len(gene_symbol)):
    for j in range(len(intersection4)):
        if(gene_symbol[i]==intersection4[j]):
            #print(intersection1[j])
            tmp.append(intersection4[j])
            rows.append(i)
            break 

In [83]:
print("\nGrouping for Natural Killer Cell")
print("Probe-Name","Gene-Symbol","Male-NS","Male-S","Female-NS","Female-S")
for i in rows:
    male_ns=all_data[i,0:12]
    male_s=all_data[i,12:24]
    female_ns=all_data[i,24:36]
    female_s=all_data[i,36:48]
    male_ns_mean=np.mean(male_ns)
    male_s_mean=np.mean(male_s)
    female_ns_mean=np.mean(female_ns)
    female_s_mean=np.mean(female_s)
    if(male_ns_mean<male_s_mean):
        male_s_up.append(gene_symbol[i])
    if(male_ns_mean>male_s_mean):
        male_s_down.append(gene_symbol[i])
    if(female_ns_mean<female_s_mean):
        female_s_up.append(gene_symbol[i])
    if(female_ns_mean>female_s_mean):
        female_s_down.append(gene_symbol[i])
    print(data[i+1][0],gene_symbol[i],male_ns_mean,male_s_mean,female_ns_mean,female_s_mean)


Grouping for Natural Killer Cell
Probe-Name Gene-Symbol Male-NS Male-S Female-NS Female-S
A_23_P151294 IFNG 5.915514025 6.710654066666668 6.606077316666666 5.854469466666665
A_23_P22232 KLRC2 9.717394458333333 10.676192666666667 10.301468541666667 9.549306850000002
A_23_P113716 HLA-C 18.476547083333333 18.442838999999996 18.48375175 18.575801916666666
A_24_P936272 HLA-C 15.186137666666667 14.983310166666664 14.905019250000002 15.305031708333336
A_23_P162486 PTPN6 12.292167374999998 11.98547275 11.932862041666667 12.233174083333331
A_23_P70539 HLA-C 16.353414833333332 16.180303000000002 16.022591416666668 16.37197258333333
A_23_P95917 HLA-C 17.984208916666663 17.97244416666667 17.967677916666666 18.203733583333335
A_23_P1473 PRF1 10.366630999999998 10.682892208333334 10.011584791666667 9.422278416666666
A_24_P298409 HLA-C 17.93432891666667 17.873328166666667 17.885603833333334 18.0351685
A_24_P326082 HLA-E 17.694454 17.707546583333336 17.5425235 17.80270308333333
A_23_P30848 HLA-E 16.1

In [84]:
print("\n****************** Final Grouping ***********************")
print("Female Smokers up genes" ,set(female_s_up))
print("Female Smokers Down genes", set(female_s_down))
print("Male Smokers up genes" ,set(male_s_up))
print("Male Smokers down genes" ,set(male_s_down))


****************** Final Grouping ***********************
Female Smokers up genes {'HLA-E', 'HLA-G', 'PTPN6', 'AOC2', 'CYP2S1', 'SULT1A1', 'HLA-C', 'PNKP', 'HNF4A'}
Female Smokers Down genes {'PRF1', 'IFNG', 'AS3MT', 'KLRC2', 'AADAC'}
Male Smokers up genes {'PRF1', 'HLA-E', 'HLA-G', 'IFNG', 'AS3MT', 'KLRC2', 'AADAC', 'HNF4A'}
Male Smokers down genes {'HLA-E', 'HLA-G', 'PTPN6', 'AOC2', 'CYP2S1', 'SULT1A1', 'HLA-C', 'PNKP', 'HNF4A'}
